## Multi-input Model

In [1]:
# build multi-input model (to predict 1-word answer from relevant background text and a user question)
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import Input

text_vocab_size = 10_000
question_vocab_size = 10_000
answer_vocab_size = 500

# input 1
text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocab_size, 64)(text_input) # the embedding arguments are backwards in the text
encoded_text = layers.LSTM(32)(embedded_text)

# input 2
question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocab_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# inputs are merged via concatination into dense layer
concatinated = layers.concatenate([encoded_text, encoded_question], axis=-1)
answer = layers.Dense(answer_vocab_size, activation='softmax')(concatinated)
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

2023-10-04 14:15:02.851773: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-04 14:15:02.851917: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [4]:
# train model on random data
import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocab_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocab_size, size=(num_samples, max_length))

answers = np.zeros((num_samples, answer_vocab_size))
for answer in answers:
  answer[np.random.randint(answer_vocab_size)] = 1

# mapping data to multiple inputs:
# 1. can use a list of np arrays in the appropriate order
# OR
# 2. can use a dictionary mapping an array to each input

# 1
#model.fit([text, question], answers, epochs=10, batch_size=128)

# 2
model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)

Epoch 1/10


2023-10-04 14:17:58.501290: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2023-10-04 14:17:59.034871: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2023-10-04 14:17:59.034999: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2023-10-04 14:18:02.811044: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2023-10-04 14:18:02.831370: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 33s 2s/step - loss: 6.2677 - acc: 0.0000e+00
Epoch 2/10
8/8 [==============================] - 6s 784ms/step - loss: 6.1794 - acc: 0.0020
Epoch 3/10
8/8 [==============================] - 6s 749ms/step - loss: 6.1214 - acc: 0.0030
Epoch 4/10
8/8 [==============================] - 6s 787ms/step - loss: 6.0307 - acc: 0.0010
Epoch 5/10
8/8 [==============================] - 6s 810ms/step - loss: 5.9273 - acc: 0.0070
Epoch 6/10
8/8 [==============================] - 6s 782ms/step - loss: 5.8407 - acc: 0.0160
Epoch 7/10
8/8 [==============================] - 5s 633ms/step - loss: 5.7476 - acc: 0.0320
Epoch 8/10
8/8 [==============================] - 4s 549ms/step - loss: 5.6675 - acc: 0.0220
Epoch 9/10
8/8 [==============================] - 4s 567ms/step - loss: 5.5963 - acc: 0.0370
Epoch 10/10
8/8 [==============================] - 5s 586ms/step - loss: 5.5558 - acc: 0.0550


## Multi-output Model

In [62]:
# build multi-input model (to predict user attributes from social media posts)
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model

vocab_size = 50_000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocab_size, 256)(posts_input)

x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

# multiple outputs
age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

# combine multi-output loss functions into one loss function
# default is to sum each loss, or a linear combination with weights
# weights help when different losses use different scales
# (could use lists or dictionary)
model.compile(optimizer='rmsprop',
              loss={'age': 'mse', 'income': 'categorical_crossentropy', 'gender': 'binary_crossentropy'},
              loss_weights={'age': 0.25, 'income': 1., 'gender': 10.})

In [63]:
# train model on random data
import numpy as np

# (these must be tuned for model to work.
# e.g. if post size is too small, convolutions are too large)
num_samples = 100
post_size = 1000

posts = np.random.randint(1, vocab_size, size=(num_samples, post_size))
age_targets = np.random.randint(10, 100, size=num_samples)
income_targets = np.random.randint(0, num_income_groups, size=num_samples)
income_targets = np.eye(num_income_groups)[income_targets]
gender_targets = np.random.randint(0, 2, size=num_samples)

model.fit(posts,
          {'age': age_targets,'income': income_targets, 'gender': gender_targets},
          epochs=10, batch_size=64)

Epoch 1/10


2023-10-05 11:15:56.636950: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x64x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x64x1x1xi1>'
loc

1/2 [==============>...............] - ETA: 14s - loss: 1028.4611 - age_loss: 4076.9375 - income_loss: 2.3018 - gender_loss: 0.6925

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x36x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x36x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceSh

2/2 [==============================] - 19s 5s/step - loss: 883.1523 - age_loss: 3484.9575 - income_loss: 2.3133 - gender_loss: 0.9600
Epoch 2/10
2/2 [==============================] - 2s 782ms/step - loss: 628.6981 - age_loss: 1718.0878 - income_loss: 9.2496 - gender_loss: 18.9927
Epoch 3/10
2/2 [==============================] - 1s 670ms/step - loss: 598.6541 - age_loss: 1949.1958 - income_loss: 5.0672 - gender_loss: 10.6288
Epoch 4/10
2/2 [==============================] - 2s 773ms/step - loss: 771.8395 - age_loss: 2075.3008 - income_loss: 8.7976 - gender_loss: 24.4217
Epoch 5/10
2/2 [==============================] - 2s 739ms/step - loss: 673.2586 - age_loss: 2521.4070 - income_loss: 2.6699 - gender_loss: 4.0237
Epoch 6/10
2/2 [==============================] - 2s 890ms/step - loss: 505.2428 - age_loss: 1699.4325 - income_loss: 3.2608 - gender_loss: 7.7124
Epoch 7/10
2/2 [==============================] - 1s 741ms/step - loss: 303.9539 - age_loss: 518.5051 - income_loss: 4.7571 - ge